In [1]:
CELL_TYPE = 'pDC'
N_GENES: int = 20
SEED = 'shap_studyID' #'disease_NOstudy' 'study_NOdisease' or 'int' or 'shap_studyID'
TEST_SPLIT_IDX: int = 1 #[0,4]

In [2]:
# Parameters
CELL_TYPE = "T_CD8_Naive"
SEED = 2
TEST_SPLIT_IDX = 1


In [3]:
N_SPLITS: int = 5
N_TRIALS: int = 50

In [4]:
import os
import sys
from pyprojroot.here import here
import pandas as pd
import anndata as ad
import numpy as np
import math
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import product
from sklearn.metrics import balanced_accuracy_score, f1_score
import optuna

import joblib
import pickle
import datetime

import collections

import xgboost
from sklearn.preprocessing import LabelEncoder

import scipy.sparse as ssp
import joblib

from dotenv import load_dotenv

In [5]:
load_dotenv()

True

# LOAD DATASET

In [6]:
train_adata = ad.read_h5ad(
    here(f'03_downstream_analysis/08_gene_importance/xgboost_external_validation/xgboost_TopN_genes/data_cellTypes/EXTERNAL_{CELL_TYPE}.filtered.log1p.h5ad')
)

In [7]:
if SEED != 'all':
    gene_subset = np.load(here(f'03_downstream_analysis/08_gene_importance/xgboost_external_validation/shap_gene_selection/gene_subsets_{N_GENES}/{CELL_TYPE}_{SEED}.npy'), allow_pickle=True)
    train_adata = train_adata[:,gene_subset]
    print(gene_subset)
elif SEED == 'all':
    print('Using all genes')
else:
    raise ValueError()

['ENSG00000117523' 'ENSG00000125384' 'ENSG00000118503' 'ENSG00000103187'
 'ENSG00000165732' 'ENSG00000139193' 'ENSG00000111640' 'ENSG00000128524'
 'ENSG00000104904' 'ENSG00000239713' 'ENSG00000075945' 'ENSG00000100911'
 'ENSG00000107223' 'ENSG00000167664' 'ENSG00000142546' 'ENSG00000136003'
 'ENSG00000182117' 'ENSG00000175203' 'ENSG00000182287' 'ENSG00000068796'
 'ENSG00000135046' 'ENSG00000148834' 'ENSG00000105835' 'ENSG00000105373'
 'ENSG00000175482' 'ENSG00000150093' 'ENSG00000090266' 'ENSG00000130066'
 'ENSG00000167283' 'ENSG00000118260' 'ENSG00000163660' 'ENSG00000156587'
 'ENSG00000177556' 'ENSG00000204472' 'ENSG00000244687' 'ENSG00000130429'
 'ENSG00000089280' 'ENSG00000127022' 'ENSG00000178719' 'ENSG00000108639'
 'ENSG00000128340' 'ENSG00000141506' 'ENSG00000171223' 'ENSG00000104964'
 'ENSG00000167863' 'ENSG00000165272' 'ENSG00000178562' 'ENSG00000171791'
 'ENSG00000132002' 'ENSG00000152234' 'ENSG00000144802' 'ENSG00000196126'
 'ENSG00000277734' 'ENSG00000145912' 'ENSG000001977

In [8]:
train_adata.shape

(37072, 110)

In [9]:
train_adata.obs.disease.unique()

['RA', 'COVID', 'healthy', 'HIV', 'cirrhosis', 'CD', 'SLE', 'sepsis']
Categories (8, object): ['CD', 'COVID', 'HIV', 'RA', 'SLE', 'cirrhosis', 'healthy', 'sepsis']

In [10]:
train_adata.obs.sampleID.unique()

['SCGT00val_I036024_T0', 'SCGT00val_I0364_T0', 'SCGT00val_I036019_T0', 'SCGT00val_I036028_T0', 'SCGT00val_I036021_T0', ..., '10XGenomics_10XHC2_T0', '10XGenomics_10XHC3_T0', '10XGenomics_10XHC5_T0', '10XGenomics_10XHC7_T0', '10XGenomics_10XHC8_T0']
Length: 86
Categories (86, object): ['10XGenomics_10XHC1_T0', '10XGenomics_10XHC2_T0', '10XGenomics_10XHC3_T0', '10XGenomics_10XHC4_T0', ..., 'Savage2021_BRISL6_T0', 'Savage2021_BRISL7_T0', 'Savage2021_PIDA_T0', 'Savage2021_PIDB_T0']

In [11]:
all_idxs = np.arange(train_adata.obs.shape[0])
left_out_splits = [s[1] for s in StratifiedGroupKFold(n_splits=N_SPLITS).split(all_idxs, train_adata.obs.disease, train_adata.obs.sampleID)]

In [12]:
TRAIN_SPLIT_IDXS = [0,1,2,3,4]
VAL_SPLIT_IDX = (TEST_SPLIT_IDX + 1) % 5
TRAIN_SPLIT_IDXS.remove(TEST_SPLIT_IDX)
TRAIN_SPLIT_IDXS.remove(VAL_SPLIT_IDX)
TRAIN_SPLIT_IDXS, VAL_SPLIT_IDX, TEST_SPLIT_IDX

([0, 3, 4], 2, 1)

In [13]:
train_idxs = np.concatenate([left_out_splits[idx] for idx in TRAIN_SPLIT_IDXS])
val_idxs = left_out_splits[VAL_SPLIT_IDX]
test_idxs = left_out_splits[TEST_SPLIT_IDX]

### SUBSET DATASET INTO TRAIN/TEST/VAL SPLITS

In [14]:
X_train = train_adata.X[train_idxs]
X_test = train_adata.X[test_idxs]
X_val = train_adata.X[val_idxs]
X_train.shape, X_test.shape, X_val.shape

((21631, 110), (8582, 110), (6859, 110))

In [15]:
y_train = train_adata.obs.iloc[train_idxs].disease.values.astype(str)
y_test = train_adata.obs.iloc[test_idxs].disease.values.astype(str)
y_val = train_adata.obs.iloc[val_idxs].disease.values.astype(str)
y_train.shape, y_test.shape, y_val.shape

((21631,), (8582,), (6859,))

In [16]:
lenc = LabelEncoder()
y_train_enc = lenc.fit_transform(y_train)
y_val_enc = lenc.transform(y_val)
y_test_enc = lenc.transform(y_test)

### GENERATE F1 

In [17]:
def custom_f1_score(y_true, y_pred):
    return -f1_score(y_true, y_pred.argmax(1), average='weighted')

In [18]:
eval_metric=custom_f1_score
eval_metric_name='custom_f1_score'

def objective(trial):
    params = {
        'n_estimators': 1500,
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 250),
        'subsample': trial.suggest_float('subsample', 0.1, 1.0),
        'colsample_bynode': trial.suggest_float('colsample_bynode', 0.1, 1.0),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 5e-1, log=True),
    }
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, f'validation_0-{eval_metric_name}')
    es_callback = xgboost.callback.EarlyStopping(20, min_delta=0.001)
    xgb = xgboost.XGBClassifier(
        eval_metric=eval_metric,
        callbacks=[pruning_callback, es_callback],
        n_jobs=5,
        **params
    )
    xgb.fit(
        X_train, 
        y_train_enc, 
        verbose=0,
        eval_set=[(X_val, y_val_enc)],
    )
    trial.set_user_attr('best_iteration', xgb.best_iteration)

    return xgb.best_score

In [19]:
sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(direction='minimize', sampler=sampler)
study.optimize(objective, n_trials=N_TRIALS, gc_after_trial=True)

[I 2025-05-15 18:03:50,700] A new study created in memory with name: no-name-525819c9-31e6-47fc-bc80-fd4a3241535c


[I 2025-05-15 18:03:51,876] Trial 0 finished with value: -0.614841 and parameters: {'max_depth': 9, 'min_child_weight': 238, 'subsample': 0.7587945476302645, 'colsample_bynode': 0.6387926357773329, 'learning_rate': 0.0026368755339723046}. Best is trial 0 with value: -0.614841.


[I 2025-05-15 18:04:03,615] Trial 1 finished with value: -0.766757 and parameters: {'max_depth': 5, 'min_child_weight': 15, 'subsample': 0.8795585311974417, 'colsample_bynode': 0.6410035105688879, 'learning_rate': 0.08148293210105287}. Best is trial 1 with value: -0.766757.


[I 2025-05-15 18:04:06,760] Trial 2 finished with value: -0.612055 and parameters: {'max_depth': 3, 'min_child_weight': 243, 'subsample': 0.8491983767203796, 'colsample_bynode': 0.29110519961044856, 'learning_rate': 0.003095566460242371}. Best is trial 1 with value: -0.766757.


[I 2025-05-15 18:04:45,562] Trial 3 finished with value: -0.726899 and parameters: {'max_depth': 6, 'min_child_weight': 77, 'subsample': 0.5722807884690141, 'colsample_bynode': 0.48875051677790415, 'learning_rate': 0.006109683510122491}. Best is trial 1 with value: -0.766757.


...

[I 2025-05-15 18:07:18,136] Trial 45 pruned. Trial was pruned at iteration 0.


[I 2025-05-15 18:07:18,530] Trial 46 pruned. Trial was pruned at iteration 1.


[I 2025-05-15 18:07:35,768] Trial 47 finished with value: -0.767499 and parameters: {'max_depth': 10, 'min_child_weight': 1, 'subsample': 0.6281366404119872, 'colsample_bynode': 0.705578517419257, 'learning_rate': 0.2795617231229411}. Best is trial 42 with value: -0.773997.


[I 2025-05-15 18:07:43,205] Trial 48 pruned. Trial was pruned at iteration 59.


[I 2025-05-15 18:07:43,518] Trial 49 pruned. Trial was pruned at iteration 0.


In [20]:
out_dir = here(f'03_downstream_analysis/08_gene_importance/xgboost_external_validation/xgboost_TopN_genes/results_{N_GENES}/study')

if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    
joblib.dump(study,os.path.join(out_dir, f'{CELL_TYPE}_{SEED}_{TEST_SPLIT_IDX}_xgboost.pkl'))

['/scratch_isilon/groups/singlecell/shared/projects/Inflammation-PBMCs-Atlas/03_downstream_analysis/08_gene_importance/xgboost_external_validation/xgboost_TopN_genes/results_20/study/T_CD8_Naive_2_1_xgboost.pkl']

In [21]:
n_estimators = int(study.best_trial.user_attrs['best_iteration']*1.2)
xgb = xgboost.XGBClassifier(
        eval_metric=eval_metric,
        n_estimators=n_estimators,
        **study.best_trial.params
    )
xgb.fit(
    ssp.vstack((X_train, X_val)), 
    np.concatenate((y_train_enc, y_val_enc)),
    verbose=1,
)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=0.10615926575700632,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False,
              eval_metric=<function custom_f1_score at 0x7f22554d7f60>,
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1637233173173614, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=8, max_leaves=None,
              min_child_weight=12, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=98, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [22]:
out_dir = here(f'03_downstream_analysis/08_gene_importance/xgboost_external_validation/xgboost_TopN_genes/results_{N_GENES}/best_model')

if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    
joblib.dump(xgb, os.path.join(out_dir, f'{CELL_TYPE}_{SEED}_{TEST_SPLIT_IDX}_xgb.json'))

['/scratch_isilon/groups/singlecell/shared/projects/Inflammation-PBMCs-Atlas/03_downstream_analysis/08_gene_importance/xgboost_external_validation/xgboost_TopN_genes/results_20/best_model/T_CD8_Naive_2_1_xgb.json']

In [23]:
df_pred_test = pd.DataFrame(dict(
    cell_id=train_adata.obs.iloc[test_idxs].index.values,
    y_true=y_test, 
    y_true_code=y_test_enc, 
    y_pred=xgb.predict(X_test))).set_index('cell_id')

In [24]:
out_dir = here(f'03_downstream_analysis/08_gene_importance/xgboost_external_validation/xgboost_TopN_genes/results_{N_GENES}/predictions')

if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    
df_pred_test.to_csv(os.path.join(out_dir, f'{CELL_TYPE}_{SEED}_{TEST_SPLIT_IDX}_pred_test.zip'))

In [25]:
metrics_dict = dict(
    BAS=balanced_accuracy_score(y_true=df_pred_test.y_true_code, y_pred=df_pred_test.y_pred), WF1=f1_score(y_true=df_pred_test.y_true_code, y_pred=df_pred_test.y_pred,average='weighted'))

/scratch_isilon/groups/singlecell/shared/conda_env/xgboost-cpu/lib/python3.12/site-packages/sklearn/metrics/_classification.py:2466: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


In [26]:
metrics_dict

{'BAS': 0.5456372495629059, 'WF1': 0.6352446714379041}

In [27]:
metrics_df = pd.DataFrame.from_dict([metrics_dict]).assign(split_idx=TEST_SPLIT_IDX, gene_set_seed=SEED, cell_type=CELL_TYPE)
metrics_df

,BAS,WF1,split_idx,gene_set_seed,cell_type
0,0.545637,0.635245,1,2,T_CD8_Naive


In [28]:
out_dir = here(f'03_downstream_analysis/08_gene_importance/xgboost_external_validation/xgboost_TopN_genes/results_{N_GENES}/metrics')

if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    
metrics_df.to_csv(os.path.join(out_dir, f'{CELL_TYPE}_{SEED}_{TEST_SPLIT_IDX}_metrics.zip'))